In [ ]:
%pip install anthropic python-dotenv

In [ ]:
# Setup with LiteLLM
import os
from anthropic import Anthropic
from dotenv import load_dotenv
import json
import sqlite3
from datetime import datetime
from typing import Dict, Any

# Load environment variables
load_dotenv()

# Initialize Anthropic client (points to LiteLLM)
client = Anthropic(
    base_url=os.environ.get("ANTHROPIC_BASE_URL"),
    api_key=os.environ.get("ANTHROPIC_AUTH_TOKEN")
)

MODEL = os.environ.get("ANTHROPIC_MODEL", "claude-sonnet-4-20250514")

print("✅ ReAct Agent - Using Claude via LiteLLM")
print(f"🔗 Endpoint: {os.environ.get('ANTHROPIC_BASE_URL')}")
print(f"🤖 Model: {MODEL}")

In [ ]:
# Initialize Database
from db_helper import init_database

print(init_database())

## 🎯 THE REACT PATTERN

**ReAct = Reasoning + Acting**

This is the MOST IMPORTANT agent pattern. ~80% of production agents use this.

### The Loop:
```
┌──────────────────────────────────────────┐
│  1. THINK: What do I need?               │
│  2. ACT: Call a tool                     │
│  3. OBSERVE: Look at the result          │
│  4. THINK: Do I have enough info?        │
│     ├─ Yes → Answer the question         │
│     └─ No → Go back to step 2            │
└──────────────────────────────────────────┘
```

### Key Differences from Simple Agent:

❌ **Simple:** One tool call, done

✅ **ReAct:** Loop until satisfied

**Let's build it!**

In [ ]:
# ReAct Agent Implementation
from db_helper import lookup_customer, check_order_status, get_customer_orders, process_refund
from db_tools import tools


def react_agent(user_message: str, max_iterations: int = 10):
    """
    ReAct Agent: Reasoning + Acting in a loop
    
    This agent will think, act, observe, and repeat until the task is complete.
    We're building the loop MANUALLY to understand how it works.
    
    Args:
        user_message: The user's question/request
        max_iterations: Maximum number of reasoning loops (safety limit)
    
    Returns:
        Final answer from the agent
    """
    
    # Initialize conversation with user's message
    messages = [
        {
            "role": "user",
            "content": user_message
        }
    ]
    
    # System prompt guides the agent's reasoning
    system_prompt = """You are a helpful customer support assistant that uses tools.
                    Think step by step using this pattern:
                    1. What information do I need to answer this question?
                    2. What tool can help me get that information?
                    3. Use the tool and observe the result
                    4. Do I have enough information now to answer?
                    5. If not, what's my next step? (repeat from step 2)
                    
                    Always explain your reasoning before using tools.
                    Be thorough - if a customer asks for "all orders", make sure to get ALL their orders.
                    If you need to refund multiple items, do them one at a time.
                    """
    
    print(f"🎯 Task: {user_message}")
    print("="*80)
    
    # THE REACT LOOP - This is what we're learning!
    for iteration in range(max_iterations):
        print(f"📍 Iteration {iteration + 1}/{max_iterations}")
        print("-"*80)
        
        # STEP 1: Agent THINKS and DECIDES what to do
        response = client.messages.create(
            model=MODEL,
            max_tokens=2048,
            system=system_prompt,
            messages=messages,
            tools=tools
        )
        
        # Show agent's REASONING (if any text)
        has_reasoning = False
        for content_block in response.content:
            if hasattr(content_block, "text") and content_block.text:
                print(f"🤔 Agent reasoning:")
                print(f"   {content_block.text}")
                has_reasoning = True
        
        if not has_reasoning:
            print("🤔 Agent is processing...")
        
        # STEP 2: Check if agent is DONE (no more tools needed)
        if response.stop_reason != "tool_use":
            print("="*80)
            print("✅ TASK COMPLETE - Agent has enough information")
            print("="*80)
            
            # Extract final answer
            final_answer = ""
            for content_block in response.content:
                if hasattr(content_block, "text"):
                    final_answer += content_block.text
            
            print(f"💬 Final Answer:\n{final_answer}")
            return final_answer
        
        # STEP 3: Add assistant's response to conversation history
        # This is CRUCIAL - the agent needs to see its own previous thoughts
        messages.append({
            "role": "assistant",
            "content": response.content
        })
        
        # STEP 4: EXECUTE the tools the agent wants to use
        tool_results = []
        for content_block in response.content:
            if content_block.type == "tool_use":
                tool_name = content_block.name
                tool_input = content_block.input
                
                print(f"🔧 Tool Call: {tool_name}")
                print(f"   Input: {json.dumps(tool_input, indent=10)}")
                
                # Execute the appropriate function
                if tool_name == "lookup_customer":
                    result = lookup_customer(**tool_input)
                elif tool_name == "check_order_status":
                    result = check_order_status(**tool_input)
                elif tool_name == "get_customer_orders":
                    result = get_customer_orders(**tool_input)
                elif tool_name == "process_refund":
                    result = process_refund(**tool_input)
                else:
                    result = {"error": f"Unknown function: {tool_name}"}
                
                print(f"   📊 Result: {json.dumps(result, indent=10)}")
                
                # Prepare result to send back to agent
                tool_results.append({
                    "type": "tool_result",
                    "tool_use_id": content_block.id,
                    "content": json.dumps(result)
                })
        
        # STEP 5: Add tool RESULTS to conversation history
        # Agent will see these results in the next iteration
        messages.append({
            "role": "user",
            "content": tool_results
        })
        
        # Loop continues... agent will think about the results and decide next step
    
    # Safety: If we hit max iterations
    print("⚠️ Max iterations reached - task may be incomplete")
    return "Task incomplete - reached maximum iterations"

In [ ]:
# Test 1 - Simple Single-Step Task
print("TEST 1: Simple Order Lookup (Should take 1 iteration)")

react_agent("Check the status of order ORD12345")

In [ ]:
# Test 2 - Two-Step Task
print("TEST 2: Customer Orders (Should take 2 iterations)")

react_agent("My customer ID is CUST001. Show me all my orders.")

In [ ]:
# Test 3 - Multi-Step Complex Task
print("TEST 3: Complex Multi-Step Task (3-4 iterations)")

react_agent("Hi, I'm customer CUST003. Show me all my orders and process a refund for any delivered items.")

## Analysis - What Just Happened?

### 🎓 WHAT YOU JUST WITNESSED

Let's break down what happened in Test 3:

#### Iteration 1:
- 🤔 **Agent thinks:** "I need to look up customer CUST003 first"
- 🔧 **Calls:** `get_customer_orders(CUST003)`
- 📊 **Gets:** List of orders

#### Iteration 2:
- 🤔 **Agent thinks:** "I see order ORD12347 is delivered, I can refund it"
- 🔧 **Calls:** `process_refund(ORD12347, reason="...")`
- 📊 **Gets:** Refund confirmation

#### Iteration 3:
- 🤔 **Agent thinks:** "I have all the info now, let me respond"
- ✅ **Provides final answer to user**

### KEY INSIGHTS:

---

1️⃣ The agent **DECIDES** when to call tools (not us!)

2️⃣ The agent **DECIDES** which tool to call (not us!)

3️⃣ The agent **DECIDES** when it's done (not us!)

4️⃣ We just build the loop and manage history

**This is what makes it "agentic" - autonomy in decision-making!**

In [ ]:
# Test 4 - Really Complex Task
print("TEST 4: Advanced Task Requiring Multiple Steps")

react_agent("""
            I'm customer CUST001. Please:
            1. Show me all my orders
            2. Tell me which ones are delivered
            3. Calculate my total spending
            4. Process refunds for all delivered items
            """)

## Experiment - Limited Iterations

### 🧪 EXPERIMENT: What happens with max_iterations=2?

Let's try the same complex task but limit iterations to 2.

Watch what happens...

In [ ]:
# Experiment - Limited Iterations
react_agent(
    "I'm CUST001. Show me all my orders and refund any delivered items.", 
    max_iterations=2
)